# **Installing Necessary Packages** 

In [1]:
!pip install gradio
!pip install transformers sentencepiece

     |████████████████████████████████| 1.7 MB 5.0 MB/s 
     |████████████████████████████████| 211 kB 45.7 MB/s 
     |████████████████████████████████| 54 kB 2.0 MB/s 
     |████████████████████████████████| 1.1 MB 40.7 MB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
     |████████████████████████████████| 84 kB 2.6 MB/s 
     |████████████████████████████████| 255 kB 38.2 MB/s 
     |████████████████████████████████| 2.0 MB 33.6 MB/s 
     |████████████████████████████████| 271 kB 38.4 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
     |████████████████████████████████| 144 kB 46.4 MB/s 
     |████████████████████████████████| 58 kB 5.0 MB/s 
     |████████████████████████████████| 10.9 MB 35.3 MB/s 
     |████████████████████████████████| 79 kB 6.8 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 61 kB 426 kB/s 
     |████████████████████████████████| 856 kB 44.4 MB/s 
     |████████████████████████

# **Importing Necessary Modules**

In [2]:
import gradio as gr
import warnings
import re

from gradio.mix import Series
from transformers import pipeline, MarianMTModel, MarianTokenizer

warnings.filterwarnings("ignore") # To ignore the warnings 

# **Cleaning Raw Data**

In [5]:
def clean_text(text):
  text = text.encode("ascii", errors="ignore").decode("ascii") 
  # if there is a newline, make it a whitespace
  text = re.sub("\n", " ", text) 
  # if there's more than 1 whitespace, make it just 1
  text = re.sub("\s+", " ", text) 
  # if there is a tab, make it just a whitespace
  text = re.sub(r"\t", " ", text)
  # removing spaces at the beginning and end of the strings
  text = text.strip(" ") 
  return text   

# **Making our first part of the Gradio Series (Defining our translation model)**

In [7]:
model_name = ("Helsinki-NLP/opus-mt-tr-en")  

# Our first function in our Gradio "Series"
def tr_to_eng(text):
    input_text = clean_text(text)

    model = MarianMTModel.from_pretrained(model_name)

    tokenizer = MarianTokenizer.from_pretrained(model_name)

    batch = tokenizer.prepare_seq2seq_batch(input_text, return_tensors="pt")

    output = model.generate(**batch)

    translated = tokenizer.batch_decode(output, skip_special_tokens=True)

    return translated[0]


# The first app to be put in "Series"
translation = gr.Interface(fn=tr_to_eng, inputs=gr.inputs.Textbox(), outputs=gr.outputs.Textbox())

# **Making our second part of Gradio Series (Defining our summarization model)**

In [8]:
pipeline_sum = pipeline(
    "summarization",
    model="facebook/bart-large-cnn",  
    tokenizer="facebook/bart-large-cnn", 
    framework="pt",
)

# The second function in our Gradio "Series"
def fb_summarizer(text):
    input_text = clean_text(text)
    results = pipeline_sum(input_text)
    return results[0]["summary_text"]


# The second app to be put in "Series"
summary = gr.Interface(fn=fb_summarizer, inputs=gr.inputs.Textbox(), outputs=gr.outputs.Textbox())

Downloading:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [9]:
Series(
    translation,
    summary,
    title="Translate And Summarize Turkish Text Into English",
    inputs=gr.inputs.Textbox(lines=20, label="Paste some Turkish text here"),
    outputs=gr.outputs.Textbox(label="English Summary"),
).launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://37839.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<fastapi.applications.FastAPI at 0x7f00775d7d90>,
 'http://127.0.0.1:7860/',
 'https://37839.gradio.app')